In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
# import pingouin as pg
import plotly.express as px
from plotly.express.colors import n_colors
import plotly.graph_objects as go
from os.path import join as pjoin
from natsort import natsort_keygen, natsorted

sys.path.append("../")
import circletrack_behavior as ctb
import plotting_functions as pf

In [ ]:
## Settings
project_folder = ['MultiCon_Imaging']
experiment_folders = ['MultiCon_Imaging5', 'MultiCon_Imaging6']
dpath = f'../../{project_folder[0]}'
fig_path = f'../../../Manuscripts/MultiCon/intermediate_plots'
chance_color = '#7d7d7d'
avg_color = '#287347'
subject_color = '#7d7d7d'
ce_colors = ['#287347', 'midnightblue']
ce_colors_dict = {'Control': '#B8293D', 'Experimental': '#287347', 'Two-Context': 'midnightblue', 'Multi-Context': '#287347'}
symbol_dict = {'Control': 'x', 'Experimental': 'circle'}
symbols_list = ['x', 'circle']
context_colors = {'A': '#00802d', 'B': '#006c79', 'C': '#004da4', 'D': '#430073',
                  'A1-5': '#00802d', 'A5-10': '#006c79', 'A10-15': '#004da4'}
mouse_colors = ['midnightblue', 'darkred', 'darkorchid', 'darkturquoise']
mf_colors = ['darkorchid', 'midnightblue']
error_color = ['rgba(153,50,204,0.4)', 'rgba(25,25,112,0.4)', 'rgba(169,169,169,0.4)'] #'rgba(169,169,169,0.4)' is gray
male_mice = ['mc44', 'mc46', 'mc54', 'mc55', 'mc57']
control_mice = ['mc46', 'mc49', 'mc52', 'mc54', 'mc57', 'mc59', 'mc60']
excluded_mice = ['mc48']
experimental_sessions = [f'A{x}' for x in np.arange(1, 6)] + [f'B{x}' for x in np.arange(1, 6)] + [f'C{x}' for x in np.arange(1, 6)] + [f'D{x}' for x in np.arange(1, 6)]
control_sessions = [f'A{x}' for x in np.arange(1, 16)] + [f'B{x}' for x in np.arange(1, 6)]
lick_thresh = 5

if not os.path.exists(fig_path):
    os.makedirs(fig_path)

### Plot weight percent change across days of the experiment.

In [ ]:
weight_data = pd.read_csv(pjoin(fig_path, 'intermediate_data/weight_percent_change.csv'))
column_list = [f'{x}' for x in np.arange(1, 28)]
weight_melt = weight_data.melt(id_vars=['Mouse'], value_vars=column_list, var_name='day', value_name='percent_change')
weight_melt['day'] = weight_melt['day'].astype(int)
avg_weight = weight_melt.groupby(['day'], as_index=False).agg({'percent_change': ['mean', 'sem']})

fig = pf.custom_graph_template(x_title='Day', y_title='Weight Change from Baseline (%)')
fig.add_trace(go.Scatter(x=natsorted(avg_weight['day']), y=avg_weight['percent_change']['mean'], mode='lines+markers', line_color=avg_color,
                         error_y=dict(type='data', array=avg_weight['percent_change']['sem']), showlegend=False))

fig.update_yaxes(range=[75, 101])
fig.add_vline(x=7.5, line_width=1, opacity=1, line_dash='dash', line_color=chance_color)
fig.add_hline(y=80, line_width=1, opacity=1, line_dash='dash', line_color=chance_color)
fig.show()
fig.write_image(pjoin(fig_path, 'weight_from_baseline.png'))

### Plot fluorescence intensity across days of the experiment.

In [ ]:
fluo_data = pd.read_csv(pjoin(fig_path, 'intermediate_data/fluorescence_data.csv'))
avg_fluo = fluo_data.groupby(['day'], as_index=False).agg({'mean_fluorescence': ['mean', 'sem']})

fig = pf.custom_graph_template(x_title='Day', y_title='Mean Fluorescence (a.u.)')
fig.add_trace(go.Scatter(x=natsorted(avg_fluo['day']), y=avg_fluo['mean_fluorescence']['mean'], mode='lines+markers', line_color=avg_color,
                         error_y=dict(type='data', array=avg_fluo['mean_fluorescence']['sem']), showlegend=False))
for mouse in fluo_data['mouse'].unique():
    mdata = fluo_data[fluo_data['mouse'] == mouse]
    fig.add_trace(go.Scatter(x=mdata['day'], y=mdata['mean_fluorescence'], mode='lines', line_color=subject_color, 
                             name=mouse, opacity=0.7, line=dict(width=1), showlegend=False))
fig.show()

### Linear track rewards across days.

In [ ]:
## Linear track behavior
data_of_interest = 'lin_behav' ## one of behav, aligned_minian, lin_behav
lin_dict = {'mouse': [], 'experiment': [], 'group': [], 'group_two': [], 'day': [], 'rewards': []}
for experiment in os.listdir(dpath):
    if experiment not in experiment_folders:
        pass 
    else:
        exp_path = pjoin(dpath, f'{experiment}/output/{data_of_interest}/')
        
        for mouse in os.listdir(exp_path):
            mpath = pjoin(exp_path, mouse)
            sex = 'Male' if mouse in male_mice else 'Female'
            group = 'Control' if mouse in control_mice else 'Experimental'
            
            for idx, session in enumerate(os.listdir(mpath)):
                behav = pd.read_feather(pjoin(mpath, session))
                lin_dict['mouse'].append(mouse)
                lin_dict['experiment'].append(behav['cohort'].unique()[0])
                lin_dict['group'].append(sex)
                lin_dict['group_two'].append(group)
                lin_dict['day'].append(idx+1)
                lin_dict['rewards'].append(np.sum(behav['water']))
lin_df = pd.DataFrame(lin_dict)

In [ ]:
## Plot rewards across days on linear track
fig = pf.plot_behavior_across_days(lin_df, x_var='day', y_var='rewards', groupby_var=['day'], plot_transitions=None,
                                   marker_color=subject_color, avg_color=avg_color, expert_line=False, chance=False,
                                   x_title='Day', y_title='Rewards', titles=['Linear Track'], height=500, width=500)
fig.show()
fig.write_image(pjoin(fig_path, 'linear_track_rewards.png'))

In [ ]:
## Plot rewards across days on linear track between male and female
fig = pf.plot_behavior_across_days(lin_df, x_var='day', y_var='rewards', groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=ce_colors, avg_color=avg_color, expert_line=False, chance=False,
                                   x_title='Day', y_title='Rewards', titles=['Linear Track'], height=500, width=500)
fig.show()
fig.write_image(pjoin(fig_path, 'linear_track_rewards_mf.png'))
lin_df.mixed_anova(dv='rewards', within='day', between='group', subject='mouse')

### Circle track lick accuracy and rewards across days.

In [ ]:
## Circle track behavior
lick_thresh = 5
data_of_interest = 'behav' ## one of behav, aligned_minian, lin_behav
circ_dict = {'mouse': [], 'experiment': [], 'sex': [], 'group': [], 'day': [], 'session': [], 'rewards': [], 'percent_correct': []}
for experiment in os.listdir(dpath):
    if experiment not in experiment_folders:
        pass 
    else:
        exp_path = pjoin(dpath, f'{experiment}/output/{data_of_interest}/')
        for mouse in os.listdir(exp_path):
            if (mouse == 'mc48') | (mouse == 'mc57'):
                pass 
            else:
                mpath = pjoin(exp_path, mouse)
                sex = 'Male' if mouse in male_mice else 'Female'
                group = 'Two-Context' if mouse in control_mice else 'Multi-Context'
                for idx, session in enumerate(os.listdir(mpath)):
                    behav = pd.read_feather(pjoin(mpath, session))
                    behav = behav[~behav['probe']] ## exclude probe
                    reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]    
                    pc = ctb.lick_accuracy(behav, port_list=[reward_one, reward_two], lick_threshold=lick_thresh, by_trials=False)
                    circ_dict['mouse'].append(mouse)
                    circ_dict['experiment'].append(behav['cohort'].unique()[0])
                    circ_dict['sex'].append(sex)
                    circ_dict['group'].append(group)
                    circ_dict['day'].append(idx+1)
                    circ_dict['session'].append(behav['session'].unique()[0])
                    circ_dict['rewards'].append(np.sum(behav['water']))
                    circ_dict['percent_correct'].append(pc)
ct_df = pd.DataFrame(circ_dict)

In [ ]:
## Plot lick accuracy for just experimental mice
fig = pf.plot_behavior_across_days(ct_df[ct_df['group'] == 'Experimental'], x_var='day', y_var='percent_correct', groupby_var=['day'], plot_transitions=[5.5, 10.5, 15.5],
                                   marker_color=subject_color, avg_color=avg_color, expert_line=False, chance=True, transition_color=['darkgrey', 'darkgrey', 'darkgrey'],
                                   plot_datapoints=False, x_title='Day', y_title='Lick Accuracy (%)', titles=['Circle Track'], height=500, width=500)
fig.update_yaxes(range=[0, 100])
fig.show()
fig.write_image(pjoin(fig_path, 'lick_accuracy_experimental_only.png'))

In [ ]:
## Plot 5th lick accuracy across days for control vs experimental
fig = pf.plot_behavior_across_days(ct_df[ct_df['day'] < 21], x_var='day', y_var='percent_correct', groupby_var=['day', 'group'], plot_transitions=[5.5, 10.5, 15.5],
                                   marker_color=ce_colors, avg_color=avg_color, expert_line=False, chance=True, transition_color=['darkgrey', 'darkgrey', 'darkgrey'],
                                   symbols=symbols_list, plot_datapoints=False, x_title='Day', y_title='Lick Accuracy (%)', titles=[''], height=500, width=600)
fig.update_yaxes(range=[0, 101])
fig.show()
fig.write_image(pjoin(fig_path, 'lick_accuracy_control_experimental.png'))

In [ ]:
tdata = ct_df[ct_df['day'] == 16]
pg.ttest(x=tdata['percent_correct'][tdata['group'] == 'Experimental'], y=tdata['percent_correct'][tdata['group'] == 'Control'])

In [ ]:
## Plot rewards across days for control and experimental
fig = pf.plot_behavior_across_days(ct_df[ct_df['day'] < 21], x_var='day', y_var='rewards', groupby_var=['day', 'group'], plot_transitions=[5.5, 10.5, 15.5],
                                   marker_color=ce_colors, avg_color=avg_color, expert_line=False, chance=False, transition_color=['darkgrey', 'darkgrey', 'darkgrey'],
                                   symbols=symbols_list, plot_datapoints=False, x_title='Day', y_title='Rewards', titles=[''], height=500, width=600)
fig.show()
fig.write_image(pjoin(fig_path, 'rewards_control_experimental.png'))

In [ ]:
## Plot lick accuracy for context A between male and female mice
fig = pf.plot_behavior_across_days(ct_df[ct_df['day'] < 6], x_var='day', y_var='percent_correct', groupby_var=['day', 'sex'],
                                   marker_color=mf_colors, avg_color=avg_color, expert_line=False, chance=True, plot_transitions=None,
                                   plot_datapoints=False, x_title='Day', y_title='Lick Accuracy (%)', titles=['Context A'], height=500, width=500)
fig.update_yaxes(range=[0, 100])
fig.show()
fig.write_image(pjoin(fig_path, 'accuracy_contextA_mf.png'))

In [ ]:
## Plot lick accuracy between male and female mice in experimental group
fig = pf.plot_behavior_across_days(ct_df[(ct_df['group'] == 'Experimental') & (ct_df['day'] < 21)], x_var='day', y_var='percent_correct', groupby_var=['day', 'sex'],
                                   marker_color=mf_colors, avg_color=avg_color, expert_line=False, chance=True, plot_transitions=[5.5, 10.5, 15.5],
                                   transition_color=['darkgrey', 'darkgrey', 'darkgrey'], symbols=symbols_list,
                                   plot_datapoints=False, x_title='Day', y_title='Lick Accuracy (%)', titles=[''], height=500, width=500)
fig.update_yaxes(range=[0, 101])
fig.show()
fig.write_image(pjoin(fig_path, 'lick_accuracy_experimental_mf.png'))

In [ ]:
## Plot lick accuracy between male and female mice in experimental group
fig = pf.plot_behavior_across_days(ct_df[ct_df['group'] == 'Two-Context'], x_var='day', y_var='percent_correct', groupby_var=['day', 'sex'],
                                   marker_color=mf_colors, avg_color=avg_color, expert_line=False, chance=True, plot_transitions=[5.5, 10.5, 15.5],
                                   transition_color=['darkgrey', 'darkgrey', 'darkgrey'], symbols=symbols_list,
                                   plot_datapoints=False, x_title='Day', y_title='Lick Accuracy (%)', titles=[''], height=500, width=500)
fig.update_yaxes(range=[0, 101])
fig.show()
fig.write_image(pjoin(fig_path, 'lick_accuracy_control_mf.png'))

In [ ]:
## Plot lick accuracy as separate lines for A, B, C, and D for experimental mice
exp_df = ct_df[(ct_df['group'] == 'Multi-Context') & (ct_df['session'] != 'AR')]
fig = pf.custom_graph_template(x_title='Day in Context', y_title='Lick Accuracy (%)')
# for session in exp_df['session'].unique():
for session in ['A','B']:
    sdata = exp_df[exp_df['session'] == session].reset_index(drop=True)
    sdata.loc[:, 'context_day'] = [1, 2, 3, 4, 5] * int(sdata.shape[0]/5)
    avg_sdata = sdata.groupby(['context_day'], as_index=False).agg({'percent_correct': ['mean', 'sem']})
    fig.add_trace(go.Scatter(x=avg_sdata['context_day'], y=avg_sdata['percent_correct']['mean'], name=session,
                             error_y=dict(type='data', array=avg_sdata['percent_correct']['sem']), line_color=context_colors[session]))
fig.add_hline(y=25, line_dash='dash', line_color=chance_color, line_width=1, opacity=1)
fig.update_yaxes(range=[0, 101])
fig.show()
fig.write_image(pjoin(fig_path, 'multi_context_AB.png'))

In [ ]:
## Plot lick accuracy as separate lines for A, B, C, and D for control mice
exp_df = ct_df[(ct_df['group'] == 'Two-Context') & (ct_df['session'] != 'AR')]
exp_df = ct_df[(ct_df['day'] < 6) & (ct_df['session'] == 'A') | (ct_df['day'] > 15) & (ct_df['session'] == 'B')]
exp_df = exp_df.replace({'B': 'D'})

fig = pf.custom_graph_template(x_title='Day in Context', y_title='Lick Accuracy (%)')
for session in ['A', 'D']:
    gdata = exp_df[exp_df['session'] == session].reset_index(drop=True)
    gdata.loc[:, 'context_day'] = [1, 2, 3, 4, 5] * int(gdata.shape[0]/5)
    avg_gdata = gdata.groupby(['context_day'], as_index=False).agg({'percent_correct': ['mean', 'sem']})
    fig.add_trace(go.Scatter(x=avg_gdata['context_day'], y=avg_gdata['percent_correct']['mean'], name=session,
                             error_y=dict(type='data', array=avg_gdata['percent_correct']['sem']), line_color=context_colors[session]))
fig.add_hline(y=25, line_dash='dash', line_color=chance_color, line_width=1, opacity=1)
fig.update_yaxes(range=[0, 100])
fig.show()
fig.write_image(pjoin(fig_path, 'two_context_AD.png'))

In [ ]:
## Plot lick accuracy as separate lines for A, B, C, and D for control mice
two_cont_list = ['A1-5', 'A1-5', 'A1-5', 'A1-5', 'A1-5', 'A5-10', 'A5-10', 'A5-10', 'A5-10', 'A5-10', 
                 'A10-15', 'A10-15', 'A10-15', 'A10-15', 'A10-15', 'D', 'D', 'D', 'D', 'D']
exp_df = ct_df[(ct_df['group'] == 'Two-Context') & (ct_df['session'] != 'AR')]
exp_df = exp_df.replace({'B': 'D'})
exp_df['session_two'] = two_cont_list * np.unique(exp_df['mouse']).shape[0]

fig = pf.custom_graph_template(x_title='Day in Context', y_title='Lick Accuracy (%)')
for session in ['A1-5', 'A5-10', 'A10-15', 'D']:
    gdata = exp_df[exp_df['session_two'] == session].reset_index(drop=True)
    gdata.loc[:, 'context_day'] = [1, 2, 3, 4, 5] * int(gdata.shape[0]/5)
    avg_gdata = gdata.groupby(['context_day'], as_index=False).agg({'percent_correct': ['mean', 'sem']})
    fig.add_trace(go.Scatter(x=avg_gdata['context_day'], y=avg_gdata['percent_correct']['mean'], name=session,
                             error_y=dict(type='data', array=avg_gdata['percent_correct']['sem']), line_color=context_colors[session]))
fig.add_hline(y=25, line_dash='dash', line_color=chance_color, line_width=1, opacity=1)
fig.update_yaxes(range=[0, 100])
fig.show()
fig.write_image(pjoin(fig_path, 'two_context_A5A10A15D.png'))

In [ ]:
## Plot lick accuracy in D for both groups
exp_df = ct_df[ct_df['session'] != 'AR']
exp_df = exp_df[exp_df['day'] > 15]
exp_df = exp_df.replace({'B': 'D'})

fig = pf.custom_graph_template(x_title='Day in Context', y_title='Lick Accuracy (%)', titles=['Context D'])
for group in ['Two-Context', 'Multi-Context']:
    gdata = exp_df[exp_df['group'] == group].reset_index(drop=True)
    gdata.loc[:, 'context_day'] = [1, 2, 3, 4, 5] * int(gdata.shape[0]/5)
    avg_gdata = gdata.groupby(['context_day'], as_index=False).agg({'percent_correct': ['mean', 'sem']})
    fig.add_trace(go.Scatter(x=avg_gdata['context_day'], y=avg_gdata['percent_correct']['mean'], name=group, mode='lines+markers',
                             line_color=ce_colors_dict[group], error_y=dict(type='data', array=avg_gdata['percent_correct']['sem'])))
fig.update_yaxes(range=[0, 100])
fig.add_hline(y=25, line_width=1, opacity=1, line_dash='dash', line_color=chance_color)
fig.show()
fig.write_image(pjoin(fig_path, 'lick_accuracy_D_multi_two.png'))

In [ ]:
## Plot lick accuracy in A just for experimental mice
fig = pf.plot_behavior_across_days(ct_df[ct_df['day'] < 6], x_var='day', y_var='percent_correct', groupby_var=['day'], plot_transitions=None,
                                   marker_color=subject_color, avg_color=avg_color, expert_line=False, chance=True, transition_color=None,
                                   plot_datapoints=False, x_title='Day', y_title='Lick Accuracy (%)', titles=['Training'], height=500, width=500)
fig.update_yaxes(range=[0, 100])
fig.show()
fig.write_image(pjoin(fig_path, 'lick_acc_in_A_all.png'))

### Example mouse linear position, lick accuracy across days.

In [ ]:
## Plot linearized position
mouse = 'mc44'
position_color = 'darkgrey'
data_of_interest = 'behav' 
fig = pf.custom_graph_template(x_title='', y_title='', rows=5, columns=1, shared_y=True, shared_x=True, 
                            titles=['Context A'], width=800, height=1000)

exp_path = pjoin(dpath, f'{experiment_folders[0]}/output/{data_of_interest}/')
mpath = pjoin(exp_path, mouse)
sex = 'Male' if mouse in male_mice else 'Female'
group = 'Control' if mouse in control_mice else 'Experimental'
for idx, session in enumerate(os.listdir(mpath)[0:5]):
    data_out = pd.read_feather(pjoin(mpath, session))
    lin_pos = data_out['a_pos']
    fig.add_trace(go.Scattergl(x=data_out['t'], y=lin_pos, mode='lines', marker_color=position_color, marker_size=2, showlegend=False), row=idx + 1, col=1)
    fig.add_trace(go.Scattergl(x=data_out['t'][data_out['water']], y=lin_pos[data_out['water']], 
                            mode='markers', marker_color='red', marker_size=6, name='Rewards'), row=idx + 1, col=1)
    fig.add_trace(go.Scattergl(x=data_out['t'][data_out['lick_port'] != -1], y=lin_pos[data_out['lick_port'] != -1],
                            mode='markers', marker_color='black', marker_size=4, opacity=0.3, name='Licks'), row=idx + 1, col=1)
fig.update_yaxes(title='Position (deg)', col=1)
fig.update_xaxes(title='Time (s)', row=5)
fig.show()
# fig.write_image(pjoin(fig_path, 'lin_pos_with_reward.png'))

In [ ]:
## Plot linearized position
mouse = 'mc44'
position_color = 'darkgrey'
data_of_interest = 'behav' 
fig = pf.custom_graph_template(x_title='Time (s)', y_title='', rows=1, columns=2, shared_y=True, shared_x=True, 
                               titles=['Day 1', 'Day 5'], width=1200, height=500)

exp_path = pjoin(dpath, f'{experiment_folders[0]}/output/{data_of_interest}/')
mpath = pjoin(exp_path, mouse)
sex = 'Male' if mouse in male_mice else 'Female'
group = 'Control' if mouse in control_mice else 'Experimental'
count = 0
for idx, session in enumerate(os.listdir(mpath)[0:5]):
    if (idx == 0) | (idx == 4):
        count += 1
        data_out = pd.read_feather(pjoin(mpath, session))
        lin_pos = data_out['a_pos']
        fig.add_trace(go.Scattergl(x=data_out['t'], y=lin_pos, mode='lines', marker_color=position_color, marker_size=2, showlegend=False), row=1, col=count)
        fig.add_trace(go.Scattergl(x=data_out['t'][data_out['water']], y=lin_pos[data_out['water']], 
                                mode='markers', marker_color='red', marker_size=10, name='Rewards', showlegend=False), row=1, col=count)
        fig.add_trace(go.Scattergl(x=data_out['t'][data_out['lick_port'] != -1], y=lin_pos[data_out['lick_port'] != -1],
                                mode='markers', marker_color='black', marker_size=6, opacity=0.6, name='Licks', showlegend=False), row=1, col=count)
fig.update_yaxes(title='Position (deg)', col=1)
fig['data'][1]['showlegend'] = True 
fig['data'][2]['showlegend'] = True 
fig.show()
fig.write_image(pjoin(fig_path, 'lin_pos_with_reward_day1_day5.png'))

In [ ]:
## Plot lick accuracy across days
mdata = ct_df[(ct_df['mouse'] == mouse) & (ct_df['day'] < 6)]
fig = pf.custom_graph_template(x_title='Day', y_title='Lick Accuracy (%)')
fig.add_trace(go.Scatter(x=mdata['day'], y=mdata['percent_correct'], mode='lines', line_color=subject_color))
fig.add_hline(y=25, line_dash='dash', line_width=1, line_color=chance_color, opacity=1)
fig.update_yaxes(range=[0, 101])
fig.show()
fig.write_image(pjoin(fig_path, f'{mouse}_example_lick_acc.png'))

In [ ]:
## Plot lick accuracy across trials across days for an example mouse. Must run code below to get dataframe
mdata = trial_df[(trial_df['mouse'] == mouse) & (trial_df['day'] < 6)]
fig = pf.custom_graph_template(x_title='', y_title='', rows=1, columns=5, shared_y=True, shared_x=True,
                                titles=['A1', 'A2', 'A3', 'A4', 'A5'], width=1000, height=300)

for day in mdata['day'].unique():
    tdata = mdata[mdata['day'] == day]
    x_data = np.arange(1, np.array(tdata['trial'])[-1]*bin_size, bin_size)
    fig.add_trace(go.Scatter(x=x_data, y=tdata['lick_acc'], mode='lines', line_color=subject_color, showlegend=False), row=1, col=day)
fig.add_hline(y=25, line_dash='dash', line_width=1, line_color=chance_color, opacity=1)
fig.update_yaxes(range=[0, 101])
fig.update_yaxes(title='Lick Accuracy (%)', col=1)
fig.update_xaxes(title='Trial')
fig.show()
fig.write_image(pjoin(fig_path, f'{mouse}_acc_across_trials.png'))

### Plot probe accuracies separately for control and experimental mice.

In [ ]:
data_of_interest = 'behav' ## one of behav, aligned_minian, lin_behav
lick_dict_probe = {'mouse': [], 'experiment': [], 'session': [], 'sex': [], 'group': [],
                   'day': [], 'num_licks': [], 'probe_acc': [], 'session_acc': []}
for experiment in os.listdir(dpath):
    if experiment not in experiment_folders:
        pass 
    else:
        exp_path = pjoin(dpath, f'{experiment}/output/{data_of_interest}/')
        for mouse in os.listdir(exp_path):
            if (mouse == 'mc48') | (mouse == 'mc57'):
                pass 
            else:
                mpath = pjoin(exp_path, mouse)
                sex = 'Male' if mouse in male_mice else 'Female'
                group = 'Control' if mouse in control_mice else 'Experimental'
                for idx, session in enumerate(os.listdir(mpath)):
                    behav = pd.read_feather(pjoin(mpath, session))
                    if any(behav['probe']):
                        behav_probe = behav[behav['probe']]
                        behav_no_probe = behav[~behav['probe']]
                        reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
                        percent_correct = ctb.lick_accuracy(behav_probe, port_list=[reward_one, reward_two], lick_threshold=lick_thresh, by_trials=False)
                        session_pc = ctb.lick_accuracy(behav_no_probe, port_list=[reward_one, reward_two], lick_threshold=lick_thresh, by_trials=False)
                        lick_dict_probe['mouse'].append(mouse)
                        lick_dict_probe['experiment'].append(behav['cohort'].unique()[0])
                        lick_dict_probe['sex'].append(sex)
                        lick_dict_probe['group'].append(group)
                        lick_dict_probe['day'].append(idx+1)
                        lick_dict_probe['session'].append(np.unique(behav['session'])[0])
                        lick_dict_probe['num_licks'].append(len(behav_probe[behav_probe['lick_port'] != -1]))
                        lick_dict_probe['probe_acc'].append(percent_correct)
                        lick_dict_probe['session_acc'].append(session_pc)
                    else:
                        pass
probe_df = pd.DataFrame(lick_dict_probe)

In [ ]:
## Plot probe performance for experimental mice
first_last = pd.DataFrame()
context_list = ['A', 'B', 'C', 'D']
df = probe_df[probe_df['group'] == 'Experimental']
for mouse in df['mouse'].unique():
    mouse_data = df[df['mouse'] == mouse]
    index_list = ctb.pick_context_day(mouse_data, day_index=0, contexts=context_list)
    index_list_two = ctb.pick_context_day(mouse_data, day_index=-1, contexts=context_list)
    sub_data = mouse_data.loc[index_list, :]
    sub_data.insert(0, 'day_type', 'First')
    sub_data_two = mouse_data.loc[index_list_two, :]
    sub_data_two.insert(0, 'day_type', 'Last')
    first_last = pd.concat([first_last, sub_data, sub_data_two])
avg_combined = first_last.groupby(['day_type', 'session'], as_index=False).agg({'probe_acc': ['mean', 'sem']})
avg_combined = avg_combined.replace({'First': '1', 'Last': '5'})

fig = pf.custom_graph_template(x_title='Day', y_title='', width=1200, rows=1, columns=len(context_list), 
                               titles=['A', 'B', 'C', 'D'], shared_x=True, shared_y=True)
for idx, session in enumerate(context_list):
    plot_data = avg_combined[avg_combined['session'] == session]
    fig.add_trace(go.Scatter(x=plot_data['day_type'], y=plot_data['probe_acc']['mean'], mode='markers',
                                error_y=dict(type='data', array=plot_data['probe_acc']['sem'], thickness=1.5, width=8), 
                                line_color=avg_color, showlegend=False), row=1, col=idx+1)
# for mouse in first_last['mouse'].unique():
#     mdata = first_last[first_last['mouse'] == mouse]
#     for idx, context in enumerate(mdata['session'].unique()):
#         pdata = mdata[mdata['session'] == context]
#         fig.add_trace(go.Scatter(x=pdata['day_type'], y=pdata['probe_acc'], mode='lines', line_color=chance_color,
#                                  line_width=1, opacity=0.7, showlegend=False, name=mouse), row=1, col=idx+1)

fig.add_hline(y=25, line_width=1, line_dash='dash', line_color='darkgrey', opacity=1)
fig.update_yaxes(title='Probe Accuracy (%)', range=[0, 101], col=1)
fig.show()
fig.write_image(pjoin(fig_path, 'probe_acc_experimental.png'))

In [ ]:
## Plot probe performance for just A
## Plot probe performance for experimental mice
first_last = pd.DataFrame()
context_list = ['A']
# df = probe_df[probe_df['group'] == 'Experimental']
df = probe_df.copy()
for mouse in df['mouse'].unique():
    mouse_data = df[df['mouse'] == mouse]
    index_list = ctb.pick_context_day(mouse_data, day_index=0, contexts=context_list)
    index_list_two = ctb.pick_context_day(mouse_data, day_index=-1, contexts=context_list)
    sub_data = mouse_data.loc[index_list, :]
    sub_data.insert(0, 'day_type', 'First')
    sub_data_two = mouse_data.loc[index_list_two, :]
    sub_data_two.insert(0, 'day_type', 'Last')
    first_last = pd.concat([first_last, sub_data, sub_data_two])
avg_combined = first_last.groupby(['day_type', 'session'], as_index=False).agg({'probe_acc': ['mean', 'sem']})
avg_combined = avg_combined.replace({'First': '1', 'Last': '5'})

fig = pf.custom_graph_template(x_title='Day', y_title='', width=500, rows=1, columns=len(context_list), 
                               titles=['Probe Accuracy'], shared_x=True, shared_y=True)
for idx, session in enumerate(context_list):
    plot_data = avg_combined[avg_combined['session'] == session]
    fig.add_trace(go.Scatter(x=plot_data['day_type'], y=plot_data['probe_acc']['mean'], mode='markers',
                                error_y=dict(type='data', array=plot_data['probe_acc']['sem'], thickness=1.5, width=8), 
                                line_color=avg_color, showlegend=False), row=1, col=idx+1)

fig.add_hline(y=25, line_width=1, line_dash='dash', line_color='darkgrey', opacity=1)
fig.update_yaxes(title='Lick Accuracy (%)', range=[0, 101], col=1)
fig.show()
fig.write_image(pjoin(fig_path, 'probe_acc_just_A.png'))

In [ ]:
## Plot probe performance for control mice
df = probe_df[probe_df['group_two'] == 'Control']
avg = df.groupby(['day', 'session'], as_index=False).agg({'probe_acc': ['mean', 'sem']})

fig = pf.custom_graph_template(x_title='Day', y_title='', width=800, rows=1, columns=2, 
                               titles=['A', 'B'], shared_x=True, shared_y=True)
for idx, session in enumerate(avg['session'].unique()):
    plot_data = avg[avg['session'] == session]
    fig.add_trace(go.Scatter(x=plot_data['day'], y=plot_data['probe_acc']['mean'],
                                error_y=dict(type='data', array=plot_data['probe_acc']['sem'], thickness=1.5, width=8), 
                                line_color=ce_colors[0], showlegend=False), row=1, col=idx+1)
for mouse in df['mouse'].unique():
    mdata = df[df['mouse'] == mouse]
    for idx, context in enumerate(mdata['session'].unique()):
        pdata = mdata[mdata['session'] == context]
        fig.add_trace(go.Scatter(x=pdata['day'], y=pdata['probe_acc'], mode='lines', line_color=chance_color,
                                 line_width=1, opacity=0.7, showlegend=False, name=mouse), row=1, col=idx+1)
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color='darkgrey', opacity=1)
fig.update_yaxes(title='Probe Accuracy (%)', range=[0, 100], col=1)
fig.show()
fig.write_image(pjoin(fig_path, 'probe_acc_control.png'))

### Plot probe accuracy for both groups on the same plot

In [ ]:
## Plot probe performance for control and experimental mice
probe_df = pd.DataFrame(lick_dict_probe)
idx_probe = (probe_df['day'] == 15) & (probe_df['group'] != 'Experimental')
probe_df = probe_df[~idx_probe] ## remove day 15 for control
first_last = pd.DataFrame()
context_list = ['A', 'B', 'C', 'D']
df = probe_df[probe_df['group'] == 'Experimental']
for mouse in df['mouse'].unique():
    mouse_data = df[df['mouse'] == mouse]
    index_list = ctb.pick_context_day(mouse_data, day_index=0, contexts=context_list)
    index_list_two = ctb.pick_context_day(mouse_data, day_index=-1, contexts=context_list)
    sub_data = mouse_data.loc[index_list, :]
    sub_data.insert(0, 'day_type', 'First')
    sub_data_two = mouse_data.loc[index_list_two, :]
    sub_data_two.insert(0, 'day_type', 'Last')
    first_last = pd.concat([first_last, sub_data, sub_data_two])

df_c = probe_df[probe_df['group'] == 'Control']
context_list = ['A', 'B']
for mouse in df_c['mouse'].unique():
    mouse_data = df_c[df_c['mouse'] == mouse]
    index_list = ctb.pick_context_day(mouse_data, day_index=0, contexts=context_list)
    index_list_two = ctb.pick_context_day(mouse_data, day_index=-1, contexts=context_list)
    sub_data = mouse_data.loc[index_list, :]
    sub_data.insert(0, 'day_type', 'First')
    sub_data_two = mouse_data.loc[index_list_two, :]
    sub_data_two.insert(0, 'day_type', 'Last')
    first_last = pd.concat([first_last, sub_data, sub_data_two])

avg = first_last.groupby(['day_type', 'session', 'group', 'day'], as_index=False).agg({'probe_acc': ['mean', 'sem']})
avg = avg.replace({'First': '1', 'Last': '5'})

In [ ]:
plot_mice = False
fig = pf.custom_graph_template(x_title='Day in Context', y_title='', width=1000, rows=1, columns=4, 
                               titles=['A', 'B', 'C', 'D'], shared_x=True, shared_y=True)
for g_idx, group in enumerate(avg['group'].unique()):
    gdata = avg[avg['group'] == group]
    for idx, session in enumerate(gdata['session'].unique()):
        if (group == 'Control') & (session == 'B'):
            idx = idx + 2
        plot_data = gdata[gdata['session'] == session]
        fig.add_trace(go.Scatter(x=plot_data['day_type'], y=plot_data['probe_acc']['mean'], name=plot_data['group'].unique()[0],
                                    error_y=dict(type='data', array=plot_data['probe_acc']['sem'], thickness=1.5, width=8), marker_symbol=symbol_dict[group],
                                    line_color=ce_colors[g_idx], showlegend=False, legendgroup=plot_data['group'].unique()[0]), row=1, col=idx+1)
if plot_mice:
    for mouse in first_last['mouse'].unique():
        mdata = first_last[first_last['mouse'] == mouse]
        for idx, context in enumerate(mdata['session'].unique()):
            pdata = mdata[mdata['session'] == context]
            fig.add_trace(go.Scatter(x=pdata['day_type'], y=pdata['probe_acc'], mode='markers', line_color=ce_colors_dict[pdata['group'].unique()[0]],
                                    line_width=1, opacity=0.7, showlegend=False, name=mouse, legendgroup=pdata['group'].unique()[0]), row=1, col=idx+1)

fig.add_hline(y=25, line_width=1, line_dash='dash', line_color='darkgrey', opacity=1)
fig.update_yaxes(title='Probe Accuracy (%)', range=[0, 101], col=1)
fig['data'][0]['showlegend'] = True
fig['data'][4]['showlegend'] = True
fig.show()
fig.write_image(pjoin(fig_path, 'probe_acc_control_and_experimental.png'))

### Plot lick accuracy across trials for the probe

In [ ]:
lick_dict = {'mouse': [], 'experiment': [], 'session': [], 'day': [], 'num_licks': [], 'probe_acc': []}
for experiment in os.listdir(dpath):
    if experiment not in experiment_folders:
        pass 
    else:
        exp_path = pjoin(dpath, f'{experiment}/output/{data_of_interest}/')
        for mouse in os.listdir(exp_path):
            mpath = pjoin(exp_path, mouse)
            for idx, session in enumerate(os.listdir(mpath)):
                behav = pd.read_feather(pjoin(mpath, session))
                if any(behav['probe']):
                    behav_probe = behav[behav['probe']]
                    behav_no_probe = behav[~behav['probe']]
                    if 'MultiCon_Imaging2' in exp_path:
                        lick_thresh = 3
                    else:
                        lick_thresh = 5
                    reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
                    percent_correct = ctb.lick_accuracy(behav_probe, port_one=reward_one, port_two=reward_two, lick_threshold=lick_thresh, by_trials=True)
                    lick_dict['mouse'].append(mouse)
                    lick_dict['experiment'].append(behav['cohort'].unique()[0])
                    lick_dict['day'].append(idx+1)
                    lick_dict['session'].append(np.unique(behav['session'])[0])
                    lick_dict['num_licks'].append(len(behav_probe[behav_probe['lick_port'] != -1]))
                    lick_dict['probe_acc'].append(percent_correct)
        else:
            pass
probe_trial_df = pd.DataFrame(lick_dict)

# first_last_trial = pd.DataFrame()
# context_list = ['A', 'B', 'C', 'D']
# for mouse in probe_trial_df['mouse'].unique():
#     mouse_data = probe_trial_df[probe_trial_df['mouse'] == mouse]
#     index_list = ctb.pick_context_day(mouse_data, day_index=0, contexts=context_list)
#     index_list_two = ctb.pick_context_day(mouse_data, day_index=-1, contexts=context_list)
#     sub_data = mouse_data.loc[index_list, :]
#     sub_data.insert(0, 'day_type', 'First')
#     sub_data_two = mouse_data.loc[index_list_two, :]
#     sub_data_two.insert(0, 'day_type', 'Last')
#     first_last_trial = pd.concat([first_last_trial, sub_data, sub_data_two])

### Plot lick accuracy across trials for each session.

In [ ]:
bin_size = 3
data_of_interest = 'behav'
trial_res = {'mouse': [], 'sex': [], 'group': [], 'day': [], 'session': [], 'session_two': [], 'trial': [], 'lick_acc': []}
for experiment in os.listdir(dpath):
    if experiment not in experiment_folders:
        pass 
    else:
        exp_path = pjoin(dpath, f'{experiment}/output/{data_of_interest}/')
        for mouse in os.listdir(exp_path):
            if mouse in excluded_mice:
                pass 
            else:
                mpath = pjoin(exp_path, mouse)
                sex = 'Male' if mouse in male_mice else 'Female'
                group = 'Control' if mouse in control_mice else 'Experimental'
                for idx, session in enumerate(os.listdir(mpath)):
                    behav = pd.read_feather(pjoin(mpath, f'{session}'))
                    reward_one, reward_two = behav['reward_one'].unique()[0], behav['reward_two'].unique()[0]
                    trial_acc = ctb.lick_accuracy(behav, port_list=[reward_one, reward_two], lick_threshold=lick_thresh, by_trials=True)
                    
                    if bin_size > 0:
                        binned_acc = ctb.bin_data(trial_acc, bin_size)
                    else:
                        binned_acc = trial_acc

                    for trial, val in enumerate(binned_acc):
                        trial_res['mouse'].append(mouse)
                        trial_res['sex'].append(sex)
                        trial_res['group'].append(group)
                        trial_res['day'].append(idx+1)
                        trial_res['session'].append(behav['session'].unique()[0])
                        trial_res['session_two'].append(behav['session_two'].unique()[0])
                        trial_res['trial'].append(trial+1)
                        trial_res['lick_acc'].append(val)
trial_df = pd.DataFrame(trial_res)
avg_acc = trial_df.groupby(['group', 'day', 'session_two', 'trial'], as_index=False).agg({'lick_acc': ['mean', 'sem']})
avg_acc_mf = trial_df.groupby(['group', 'day', 'session_two', 'sex', 'trial'], as_index=False).agg({'lick_acc': ['mean', 'sem']})

In [ ]:
## Plot trial accuracy for each day for experimental mice
fig = pf.custom_graph_template(x_title='', y_title='', rows=4, columns=5, titles=experimental_sessions, height=1000, width=1000,
                               shared_y=True, shared_x=True)
for group in avg_acc['group'].unique():
    if group == 'Control':
        pass 
    else:
        gdata = avg_acc[avg_acc['group'] == group]
        for idx, day in enumerate(gdata['day'].unique()):
            if idx == 20:
                pass
            else:
                day_data = gdata[gdata['day'] == day]

                if idx < 5:
                    row, col = 1, idx + 1
                elif (idx >= 5) & (idx < 10):
                    row, col = 2, idx - 4
                elif (idx >= 10) & (idx < 15):
                    row, col = 3, idx - 9
                else:
                    row, col = 4, idx - 14

                x_data = np.arange(1, np.array(day_data['trial'])[-1]*bin_size, bin_size)
                upper = day_data['lick_acc']['mean'] + day_data['lick_acc']['sem']
                lower = day_data['lick_acc']['mean'] - day_data['lick_acc']['sem']

                fig.add_trace(go.Scatter(x=x_data, y=day_data['lick_acc']['mean'], mode='lines', line_color=ce_colors_dict[group], showlegend=False), row=row, col=col)
                fig.add_trace(go.Scatter(x=x_data, y=upper, mode='lines', marker=dict(color=error_color[1]),
                                        name='Upper Bound', line=dict(width=0), showlegend=False), row=row, col=col)
                fig.add_trace(go.Scatter(x=x_data, y=lower, mode='lines', marker=dict(color=error_color[1]),
                                        name='Lower Bound', line=dict(width=0), showlegend=False, fillcolor=error_color[1], fill='tonexty'), row=row, col=col)
                             
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='Lick Accuracy (%)', col=1)
fig.update_xaxes(title='Trial', row=4)
fig.update_yaxes(range=[0, 100])
fig.show()
fig.write_image(pjoin(fig_path, 'acc_trials_experimental_all_days.png'))

In [ ]:
## Plot trial accuracy for each day for control mice
fig = pf.custom_graph_template(x_title='', y_title='', rows=4, columns=5, titles=control_sessions, height=1000, width=1000,
                               shared_y=True, shared_x=True)
for group in avg_acc['group'].unique():
    if group == 'Experimental':
        pass 
    else:
        gdata = avg_acc[avg_acc['group'] == group]
        for idx, day in enumerate(gdata['day'].unique()):
            if idx == 20:
                pass 
            else:
                day_data = gdata[gdata['day'] == day]

                if idx < 5:
                    row, col = 1, idx + 1
                elif (idx >= 5) & (idx < 10):
                    row, col = 2, idx - 4
                elif (idx >= 10) & (idx < 15):
                    row, col = 3, idx - 9
                else:
                    row, col = 4, idx - 14

                x_data = np.arange(1, np.array(day_data['trial'])[-1]*bin_size, bin_size)
                upper = day_data['lick_acc']['mean'] + day_data['lick_acc']['sem']
                lower = day_data['lick_acc']['mean'] - day_data['lick_acc']['sem']

                fig.add_trace(go.Scatter(x=x_data, y=day_data['lick_acc']['mean'], mode='lines', line_color=ce_colors_dict[group], showlegend=False), row=row, col=col)
                fig.add_trace(go.Scatter(x=x_data, y=upper, mode='lines', marker=dict(color=error_color[2]),
                                        name='Upper Bound', line=dict(width=0), showlegend=False), row=row, col=col)
                fig.add_trace(go.Scatter(x=x_data, y=lower, mode='lines', marker=dict(color=error_color[2]),
                                        name='Lower Bound', line=dict(width=0), showlegend=False, fillcolor=error_color[2], fill='tonexty'), row=row, col=col)
                             
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.update_yaxes(title='Lick Accuracy (%)', col=1)
fig.update_xaxes(title='Trial', row=4)
fig.update_yaxes(range=[0, 100])
fig.show()
fig.write_image(pjoin(fig_path, 'acc_trials_control_all_days.png'))

### Signal detection metrics.

In [ ]:
## Correct rejection rate
data_of_interest = 'behav' ## one of behav, aligned_minian, lin_behav
sig_df = pd.DataFrame()
for experiment in os.listdir(dpath):
    if experiment not in experiment_folders:
        pass 
    else:
        exp_path = pjoin(dpath, f'{experiment}/output/{data_of_interest}/')
        for mouse in os.listdir(exp_path):
            mpath = pjoin(exp_path, mouse)
            for idx, session in enumerate(os.listdir(mpath)):
                behav = pd.read_feather(pjoin(mpath, session))
                behav = behav[~behav['probe']] ## exclude probe
                reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]    
                signal = pd.DataFrame(ctb.dprime_metrics(behav, mouse, day=idx+1, reward_ports=[reward_one, reward_two], forward_reverse='forward'))
                signal['experiment'] = behav['cohort'].unique()[0]
                sig_df = pd.concat([sig_df, signal], ignore_index=True)

In [ ]:
## Plot correct rejection rate
corr_rej = sig_df.groupby(['day', 'mouse'], as_index=False).agg({'CR': 'mean'})
fig = pf.plot_behavior_across_days(corr_rej, x_var='day', y_var='CR', groupby_var=['day'], plot_transitions=[5.5, 10.5, 15.5],
                                   marker_color=subject_color, avg_color=avg_color, expert_line=False, chance=False, transition_color=['darkgrey', 'darkgrey', 'darkgrey'],
                                   x_title='Day', y_title='Correct Rejection Rate', titles=['Circle Track'], height=500, width=500)
fig.update_yaxes(range=[0, 1])
fig.show()

### Determine where mice are licking across each session.

In [ ]:
data_of_interest = 'behav'
threshold_list = [1, 2, 3, 4, 5]
lick_dict = {'mouse': [], 'day': [], 'group': [], 'group_two': [], 'session_two': [], 'lick_thresh': [], 'reward_ports': [],
             'front_ports': [], 'back_ports': [], 'final_ports': []}
for experiment in os.listdir(dpath):
    if experiment not in experiment_folders:
        pass 
    else:
        exp_path = pjoin(dpath, f'{experiment}/output/{data_of_interest}/')
        for mouse in os.listdir(exp_path):
            mpath = pjoin(exp_path, mouse)
            sex = 'Male' if mouse in male_mice else 'Female'
            group = 'Control' if mouse in control_mice else 'Experimental'
            for idx, session in enumerate(os.listdir(mpath)):
                day = idx + 1
                behav = pd.read_feather(pjoin(mpath, f'{session}'))
                behav = behav[~behav['probe']]
                reward_one, reward_two = behav['reward_one'].unique()[0], behav['reward_two'].unique()[0]
                front_ports, back_ports = ctb.front_back_ports(reward_list=[reward_one, reward_two])

                licks = behav[behav['lick_port'] != -1]

                for lick_threshold in threshold_list:
                    count = 0
                    lick_port = np.nan
                    if licks.empty:
                        pass
                    else:
                        for idx, _ in licks.iterrows():
                            if lick_port != licks.loc[idx, 'lick_port']:
                                count = 1
                            else:
                                count += 1
                            
                            if count < lick_threshold - 1:
                                licks.loc[idx, 'threshold_reached'] = False
                            elif count == lick_threshold:
                                licks.loc[idx, 'threshold_reached'] = True
                            else:
                                licks.loc[idx, 'threshold_reached'] = False

                            lick_port =  licks.loc[idx, 'lick_port']

                        port_licks = licks[['lick_port', 'threshold_reached']].groupby(['lick_port'], as_index=False).agg({'threshold_reached': 'sum'})
                        total_licks = port_licks['threshold_reached'].sum()
                        reward_port_licks = 0
                        front_port_licks = 0
                        back_port_licks = 0
                        final_port_licks = 0
                        for _, row in port_licks.iterrows():
                            p_num = row['lick_port']
                            if p_num in [reward_one, reward_two]:
                                reward_port_licks = reward_port_licks + row['threshold_reached']
                            elif p_num in front_ports:
                                front_port_licks = front_port_licks + row['threshold_reached']
                            elif p_num in back_ports:
                                back_port_licks = back_port_licks + row['threshold_reached']
                            else:
                                final_port_licks = final_port_licks + row['threshold_reached']

                        lick_dict['mouse'].append(mouse)
                        lick_dict['day'].append(day)
                        lick_dict['group'].append(sex)
                        lick_dict['group_two'].append(group)
                        lick_dict['session_two'].append(behav['session_two'].unique()[0])
                        lick_dict['lick_thresh'].append(lick_threshold)
                        lick_dict['reward_ports'].append((reward_port_licks / total_licks) * 100)
                        lick_dict['front_ports'].append((front_port_licks / total_licks) * 100)
                        lick_dict['back_ports'].append((back_port_licks / total_licks) * 100)
                        lick_dict['final_ports'].append((final_port_licks / total_licks) * 100)
lick_df = pd.DataFrame(lick_dict)

In [ ]:
## For experimental mice
lick_thresh = 5
x_axis = ['RP', 'FP', 'BP', 'LP']
fig = pf.custom_graph_template(x_title='', y_title='', height=1000, width=1000, 
                               shared_y=True, rows=4, columns=5, titles=session_list)
exp_licks = lick_df[lick_df['group_two'] == 'Experimental']
avg_licks = exp_licks.groupby(['session_two', 'lick_thresh'], as_index=False).agg({'reward_ports': ['mean', 'sem'], 'front_ports': ['mean', 'sem'],
                                                                                 'back_ports': ['mean', 'sem'], 'final_ports': ['mean', 'sem']})
for idx, session in enumerate(avg_licks['session_two'].unique()):
    pdata = avg_licks[(avg_licks['session_two'] == session) & (avg_licks['lick_thresh'] == lick_thresh)]

    if idx < 5:
        row, col = 1, idx + 1
    elif (idx >= 5) & (idx < 10):
        row, col = 2, idx - 4
    elif (idx >= 10) & (idx < 15):
        row, col = 3, idx - 9
    elif idx >= 15:
        row, col = 4, idx - 14

    y_data = [pdata['reward_ports']['mean'].values[0],
              pdata['front_ports']['mean'].values[0],
              pdata['back_ports']['mean'].values[0],
              pdata['final_ports']['mean'].values[0]]
    y_sem = [pdata['reward_ports']['sem'].values[0],
              pdata['front_ports']['sem'].values[0],
              pdata['back_ports']['sem'].values[0],
              pdata['final_ports']['sem'].values[0]]
    fig.add_trace(go.Bar(x=x_axis, y=y_data, showlegend=False, marker_color=avg_color,
                         error_y=dict(type='data', array=y_sem), marker_line_color='black', marker_line_width=2), row=row, col=col)
    
    for mouse in exp_licks['mouse'].unique():
        data = exp_licks[(exp_licks['mouse'] == mouse) & (exp_licks['session_two'] == session) & (exp_licks['lick_thresh'] == lick_thresh)]
        fig.add_trace(go.Scatter(x=x_axis, y=data.loc[:, ['reward_ports', 'front_ports', 'back_ports', 'final_ports']].to_numpy()[0],
                                 mode='markers', marker_color='darkgrey', marker=dict(line=dict(width=1)),
                                 name=mouse, showlegend=False, opacity=0.6), row=row, col=col)
fig.update_yaxes(title='5th Licks (%)', col=1, range=[0, 100])
fig.show()
fig.write_image(pjoin(fig_path, 'front_back_leftover_ports_experimental.png'))

In [ ]:
## For control mice
lick_thresh = 5
x_axis = ['RP', 'FP', 'BP', 'LP']
natsort_key = natsort_keygen()
fig = pf.custom_graph_template(x_title='', y_title='', height=1000, width=1000, 
                               shared_y=True, rows=4, columns=5, titles=control_list)
exp_licks = lick_df[lick_df['group_two'] == 'Control']
avg_licks = exp_licks.groupby(['session_two', 'lick_thresh'], as_index=False).agg({'reward_ports': ['mean', 'sem'], 'front_ports': ['mean', 'sem'],
                                                                                 'back_ports': ['mean', 'sem'], 'final_ports': ['mean', 'sem']})
for idx, session in enumerate(sorted(avg_licks['session_two'].unique(), key=natsort_key)):
    pdata = avg_licks[(avg_licks['session_two'] == session) & (avg_licks['lick_thresh'] == lick_thresh)]

    if idx < 5:
        row, col = 1, idx + 1
    elif (idx >= 5) & (idx < 10):
        row, col = 2, idx - 4
    elif (idx >= 10) & (idx < 15):
        row, col = 3, idx - 9
    elif idx >= 15:
        row, col = 4, idx - 14

    y_data = [pdata['reward_ports']['mean'].values[0],
              pdata['front_ports']['mean'].values[0],
              pdata['back_ports']['mean'].values[0],
              pdata['final_ports']['mean'].values[0]]
    y_sem = [pdata['reward_ports']['sem'].values[0],
              pdata['front_ports']['sem'].values[0],
              pdata['back_ports']['sem'].values[0],
              pdata['final_ports']['sem'].values[0]]
    fig.add_trace(go.Bar(x=x_axis, y=y_data, showlegend=False, marker_color=ce_colors[0],
                         error_y=dict(type='data', array=y_sem), marker_line_color='black', marker_line_width=2), row=row, col=col)
    
    for mouse in exp_licks['mouse'].unique():
        data = exp_licks[(exp_licks['mouse'] == mouse) & (exp_licks['session_two'] == session) & (exp_licks['lick_thresh'] == lick_thresh)]
        fig.add_trace(go.Scatter(x=x_axis, y=data.loc[:, ['reward_ports', 'front_ports', 'back_ports', 'final_ports']].to_numpy()[0],
                                 mode='markers', marker_color='darkgrey', marker=dict(line=dict(width=1)),
                                 name=mouse, showlegend=False, opacity=0.6), row=row, col=col)
fig.update_yaxes(title='5th Licks (%)', col=1, range=[0, 100])
fig.show()
fig.write_image(pjoin(fig_path, 'front_back_leftover_ports_control.png'))

### During probe on first day in new context, calculate lick accuracy based on rotationally equivalent ports.

In [ ]:
data_of_interest = 'behav' ## one of behav, aligned_minian, lin_behav
lick_dict_probe = {'mouse': [], 'experiment': [], 'session': [], 'sex': [], 'group': [], 'day': [], 'num_licks': [], 'shift': [], 'probe_acc': []}
reward_list = [x for x in np.arange(1, 9)]

for experiment in os.listdir(dpath):
    if experiment not in experiment_folders:
        pass 
    else:
        exp_path = pjoin(dpath, f'{experiment}/output/{data_of_interest}/')
        for mouse in os.listdir(exp_path):
            if (mouse == 'mc48') | (mouse == 'mc57'):
                pass 
            else:
                mpath = pjoin(exp_path, mouse)
                sex = 'Male' if mouse in male_mice else 'Female'
                group = 'Control' if mouse in control_mice else 'Experimental'

                for idx, session in enumerate(os.listdir(mpath)):
                    behav = pd.read_feather(pjoin(mpath, session))
                    if session in [f'{mouse}_5.feat', f'{mouse}_10.feat', f'{mouse}_15.feat']:
                        reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]

                    if session in [f'{mouse}_6.feat', f'{mouse}_11.feat', f'{mouse}_16.feat']:
                        behav_probe = behav[behav['probe']]
                        for shift in np.arange(1, 9):
                            shifted_rewards = np.roll(reward_list, shift)
                            percent_correct = ctb.lick_accuracy(behav_probe, port_list=[shifted_rewards[reward_one-1], shifted_rewards[reward_two-1]], 
                                                                lick_threshold=lick_thresh, by_trials=False, to_percent=True)
                            lick_dict_probe['mouse'].append(mouse)
                            lick_dict_probe['experiment'].append(behav['cohort'].unique()[0])
                            lick_dict_probe['sex'].append(sex)
                            lick_dict_probe['group'].append(group)
                            lick_dict_probe['day'].append(idx+1)
                            lick_dict_probe['session'].append(np.unique(behav['session'])[0])
                            lick_dict_probe['num_licks'].append(len(behav_probe[behav_probe['lick_port'] != -1]))
                            lick_dict_probe['shift'].append(shift)
                            lick_dict_probe['probe_acc'].append(percent_correct)
shifted_df = pd.DataFrame(lick_dict_probe)
shifted_df = shifted_df[shifted_df['group'] == 'Experimental'] ## select only the experimental group for this analysis
shifted_avg = shifted_df.groupby(['day', 'shift'], as_index=False).agg({'probe_acc': ['mean', 'sem']})
mouse_avg = shifted_df.groupby(['mouse', 'shift'], as_index=False).agg({'probe_acc': ['mean', 'sem']})

In [ ]:
## Plot avg+sem of rotations of reward ports for each day
color_dict = {6: 'darkgrey', 11: 'midnightblue', 16: 'darkorchid'}
fig = pf.custom_graph_template(x_title='Rotation Number', y_title='Lick Accuracy (%)')
for day in shifted_avg['day'].unique():
    data = shifted_avg[shifted_avg['day'] == day]
    fig.add_trace(go.Scatter(x=data['shift'], y=data['probe_acc']['mean'], name=f'Day {day}', mode='markers',
                             error_y=dict(type='data', array=data['probe_acc']['sem']), line_color=color_dict[day]))

fig.add_hline(y=25, line_width=1, opacity=1, line_color='black', line_dash='dash')
fig.update_yaxes(range=[0, 101])
fig.show()
fig.write_image(pjoin(fig_path, 'rotated_probe_accuracy_all_rotations.png'))

In [ ]:
## Violin plot of all rotations
fig = pf.custom_graph_template(x_title='Rotation Number', y_title='Lick Accuracy (%)', width=1300)
for day in shifted_df['day'].unique():
    data = shifted_df[shifted_df['day'] == day]
    fig.add_trace(go.Violin(x=data['shift'], y=data['probe_acc'], legendgroup=str(day), scalegroup=str(day), name=f'Day {day}', 
                            line_color=color_dict[day], box_visible=True, meanline_visible=True))
fig.update_traces(points='all', jitter=0.4)
fig.update_layout(violinmode='group')
fig.add_hline(y=25, line_width=1, opacity=1, line_color='black', line_dash='dash')
fig.show()

In [ ]:
## Accuracy for each rotation where the three days are collapsed into one value for each mouse
fig = pf.custom_graph_template(x_title='Rotation Number', y_title='Lick Accuracy (%)', width=1000)
fig.add_trace(go.Violin(x=mouse_avg['shift'], y=mouse_avg['probe_acc']['mean'], box_visible=True, meanline_visible=True, line_color=ce_colors_dict['Experimental']))
fig.update_traces(points='all', jitter=0.4)
fig.add_hline(y=25, line_width=1, opacity=1, line_color='black', line_dash='dash')
fig.update_yaxes(range=[-20, 101])
fig.show()

### Rotated ports, but only for the rotation based on old maze to new maze.

In [ ]:
data_of_interest = 'behav' ## one of behav, aligned_minian, lin_behav
lick_dict = {'mouse': [], 'experiment': [], 'session': [], 'sex': [], 'group': [], 'day': [], 'num_licks': [], 'probe_acc': []}

for experiment in os.listdir(dpath):
    if experiment not in experiment_folders:
        pass 
    else:
        exp_path = pjoin(dpath, f'{experiment}/output/{data_of_interest}/')
        for mouse in os.listdir(exp_path):
            if (mouse == 'mc48') | (mouse == 'mc57'):
                pass 
            else:
                mpath = pjoin(exp_path, mouse)
                sex = 'Male' if mouse in male_mice else 'Female'
                group = 'Control' if mouse in control_mice else 'Experimental'

                if group == 'Control':
                    pass
                else:

                    for idx, session in enumerate(os.listdir(mpath)):
                        behav = pd.read_feather(pjoin(mpath, session))
                        if session in [f'{mouse}_5.feat', f'{mouse}_10.feat', f'{mouse}_15.feat']:
                            reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
                            old_maze = behav['maze'].unique()[0]

                        if session in [f'{mouse}_6.feat', f'{mouse}_11.feat', f'{mouse}_16.feat']:
                            behav_probe = behav[behav['probe']]

                            ## Rotate ports
                            rot_one, rot_two = ctb.rotate_ports(input_maze=old_maze, 
                                                                output_maze=np.unique(behav_probe['maze'])[0], 
                                                                reward_one=reward_one, 
                                                                reward_two=reward_two)
                            percent_correct = ctb.lick_accuracy(behav_probe, port_list=[rot_one, rot_two], 
                                                                lick_threshold=5, by_trials=False, to_percent=True)
                            lick_dict['mouse'].append(mouse)
                            lick_dict['experiment'].append(behav['cohort'].unique()[0])
                            lick_dict['sex'].append(sex)
                            lick_dict['group'].append(group)
                            lick_dict['day'].append(idx+1)
                            lick_dict['session'].append(np.unique(behav['session'])[0])
                            lick_dict['num_licks'].append(len(behav_probe[behav_probe['lick_port'] != -1]))
                            lick_dict['probe_acc'].append(percent_correct)
rotation_df = pd.DataFrame(lick_dict)

In [ ]:
## Check if mice are using navigating based on a rotation of the previous environment
rotation_df['day'] = rotation_df['day'].replace({6: 'A to B', 11: 'B to C', 16: 'C to D'})
fig =  pf.custom_graph_template(x_title='', y_title='Lick Accuracy (%)')
fig.add_trace(go.Violin(x=rotation_df['day'], y=rotation_df['probe_acc'], box_visible=True, meanline_visible=True, line_color=ce_colors_dict['Experimental']))
fig.update_traces(points='all', jitter=0.4)
fig.add_hline(y=25, line_width=1, opacity=1, line_color='black', line_dash='dash')
fig.show()
fig.write_image(pjoin(fig_path, 'rotated_probe_oldmaze_newmaze.png'))